In [ ]:
#Données disponibles ici: https://www.kaggle.com/datasets/retailrocket/ecommerce-dataset/data

# Projet E-commerce: Comprendre le comportement des utilisateurs du site et prédire leurs comportements futurs


**Comment naviguent-ils sur le site?**

**Quelle est la proportion de produits achetés/vus/ajoutés au panier
Observons nous des pics d'acahat sur un mois en particulier?**

**Quelles sont les catégories de produits qui ont le plus d'event (vu/ajoutés/achetés)**

**Découper les acheteurs en clusters (grâce à la méthode Kmeans)**

**Déterminer le groupe susceptible de faire des transactions**

**Identifier les catégories de produits qu'ils sont susceptible acheter**

# Importation des packages nécessaires

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline
import plotly.graph_objects as go
import plotly.express as px


# Exploration des datasets
**Fichier 1: item_properties_part1** : 10999999 lignes, 4 colonnes

In [ ]:
#Import et affichage du jeu de données item_properties_part1
path="C:/Users/miche/OneDrive/Documents/Mes Projets Git hub/Bases projets"
df_it_prop1= pd.read_csv(path+"/item_properties_part1.csv")
display(df_it_prop1.head(5))
print(df_it_prop1.shape)

**Fichier 2: item_properties_part2**: 9275903 lignes, 4 colonnes

In [ ]:
#Import et affichage du jeu de données item_properties_part2
path="C:/Users/miche/OneDrive/Documents/Mes Projets Git hub/Bases projets"
df_it_prop2= pd.read_csv(path+"/item_properties_part2.csv")
display(df_it_prop2.head(5))
print(df_it_prop2.shape)

*Description des colonnes du jeu de données item_properties_part1 et 2*
1. **Timesstamp :** variable timestamp qui renseigne sur la date et l'horaire exacte.
2. **itemidid :** l'ID de l'item.
3. **property :** la propriété concernée soit la catégorie de produit surlequel le client a cliqué, soit la disponibilité du produit ect...
4. **value :** la valeur affecté au properties (Ex: le nom de la catégorie du produit). Précisons que les données catégories ont été anonymisées. Donc nous n'avons que des chiffres comme catégorie

**Concaténation de ces 2 fichiers qui ont la même structure**

In [ ]:
# concaténation des 2 tables properties
item=pd.concat([df_it_prop1,df_it_prop2], axis=0)

**Nettoyage du dataframe concaténé**:

*Vérification des valeurs manquantes*

*Vérification des doublons*

In [ ]:
# Nbre de lignes et vérification valeurs manquantes et doublons
print("Nombre de lignes dans item:", item.shape[0])
print("Pourcentage de valeurs manquantes dans les variables de item")
display(pd.DataFrame(index=item.columns, columns=['%_valeurs_manquantes'], data= (item.isna().sum().values / len(item)*100)))
print("Doublons")
display(item.duplicated().sum())

**Fichier 3: events** : 275 610 lignes, 5 colonnes

In [ ]:
df= pd.read_csv(path+"/events.csv")
display(df.head(5))
display(len(df))

*Description des colonnes du jeu de données events*
1. **Timesstamp :** variable timestamp qui renseigne sur la date et l'horaire exacte.
2. **visitordid :** l'ID des visiteurs du site.
3. **event :** l'évenement concernéé: view/ addcart/transaction
4. **iitemdid :** l'ID de l'evenement
5. **transactionid :** décrit l'id de la transaction quand une transaction été réalisé

**A ce niveau de l'exploration des fichiers de données, le fichier events nous parait plus pertinent pour la suite de l'analyse, nous allons donc récupérer les informations essentielles du dataframe item et nous allons les mettre dans le dataframe df(event)**
**Pour celà, il nous faut d'abord parcours les 2 listes d'items unique pour récupérer les items qu'ils ont en commun**. Nous mettrons ces items dans une nouvelle liste nommée liste_itemid_commun

**Récupération des items uniques du dataframe item** dans une variable nommée "liste_itemid_item"

In [ ]:
#Liste item et nbre d'itemid dans item
liste_itemid_item=item.itemid.unique()
display(liste_itemid_item)
print("Nbre itemid dans item")
display(len(liste_itemid_item))

**Récupération des items uniques du dataframe df** dans une variable nommée liste_itemid_df

In [ ]:
#Liste d'item et nbre d'itemid dans df
liste_itemid_df=df['itemid'].unique()
display(liste_itemid_df)
print("Nbre itemid dans item")
display(len(liste_itemid_df))

**Parcours des 2 listes d'items pour récupérer la liste d'items commune**

In [ ]:
#rechercher les itemsid qui sont dans item mais egalement dans df
#liste_itemid_commun=[]
#for i in liste_itemid_item:
  #for j in liste_itemid_df:
    #if i == j:
      #liste_itemid_commun.append(i)

#print(liste_itemid_commun)
#len(liste_itemid_commun)

**Filtrage de df en fonction des items contenus dans la liste commune**

In [ ]:
#filtrage de df avec les itemid commun à item et df
df=df.loc[df['itemid'].isin(liste_itemid_item)]
df=df.reset_index(drop=True)
display(len(df))

**Seules les informations sur la disponibilité ou non du produit ainsi que la catégorie du produit peuvent nous être utiles dans item** 

*Nous allons donc filtré le jeu de donnée item en fonction de 'available'et de 'categoryid' de la colonne item.property*

*Ensuite fusionner toutes ces informations avec le jeu de donnée df(events)*

In [ ]:
#création de 2 df item_ availability et item_categ
item_availability=item.loc[item.property=='available']
item_category=item.loc[item.property=='categoryid']

In [ ]:
item_category=item_category.reset_index(drop=True)
item_availability=item_availability.reset_index(drop=True)
display(item_availability.head())
display(item_category.head())

In [ ]:
display(len(item_category))
display(len(item_availability))

**Utilisation de merge_asof pour récuprer les informations sur l'évenement d'une part et les propriétés des produits d'autre part au moment le plus précis où l'evement s'est déroulé**

In [ ]:
#fusion de df avec item_availability pour récupérer les infos sur la disponibilité de nos produits
#nous utiliserons ici merge asof qui nous permettra de récupérer la disponibilité du produit à peu près au moment où l'event a été réalisé
#création merged_1

item_availability.itemid = item_availability.itemid.astype('int64')

merged_1=pd.merge_asof(df.sort_values('timestamp'),item_availability.sort_values('timestamp'),by='itemid', on='timestamp',direction='nearest')
merged_1.head()


In [ ]:
#création merged_2 pour récupérer à présent les categories de certains produits

item_category.itemid = item_category.itemid.astype('int64')

merged_2=pd.merge_asof(merged_1.sort_values('timestamp'),item_category.sort_values('timestamp'),by='itemid', on='timestamp',direction='nearest')
merged_2.head()

**Le df nommé merged_2 comporte ainsi toutes les informations récupérer du df item et du df(event)**

**Nous pouvons donc procéder à son nettoyage également**

In [ ]:
#suppression des colonnes property et transactionid
merged_2=merged_2.drop(['property_x','property_y','transactionid'], axis=1)

In [ ]:
#renommer les colonnes pour plus de clarté
merged_2=merged_2.rename(columns={'value_x': "available", "value_y": "categoryid"})
merged_2.head()

In [ ]:
merged_2.isna().sum()

In [ ]:
merged_2.categoryid=merged_2.categoryid.astype(int)
merged_2.info()

**Fichier 4: category** : 275 610 lignes, 5 colonnes

In [ ]:
tree= pd.read_csv(path+"/category_tree.csv")
display(tree.head(5))
display(len(tree))

*Description des colonnes du jeu de données category*
1. **categoryid :** la catégorie du produit
2. **parentid:** la catégorie supérieure (parent) auxquelle est rattaché la catégorie .

*Nettoyage du jeu de données*

In [ ]:
#Nbre de lignes et valeurs manquantes category
print("Nombre de lignes dans category_tree:", tree.shape[0])
print("Pourcentage de valeurs manquantes dans les variables de category_tree")
pd.DataFrame(index=tree.columns, columns=['%_valeurs_manquantes'], data= (tree.isna().sum().values / len(tree)*100))

In [ ]:
# Vérification de doublons
tree.duplicated().sum()

**- Dernière fusion du df merged_2 avec ce 4ième fichier pour récupérer les parentid**

**- Suite du pré processing:**

1- Création de 3 nouvelles colonnes issues de la colonne *event* pour avoir une colonne par vus/Ajout au panier/transaction

2- Conversion de timestamp en datetime

3- Création d'une colonne month

4- Vérification et dernièrs nettoyages

In [ ]:
#nous allons à présent récuperer les informations sur les parentid correspondant à nos catégories dans tree
df_final=merged_2.merge(tree, how='left', on= 'categoryid')

In [ ]:
#création de 3 nouvelles variables à partir de la colonne event
df_final=df_final.join(pd.get_dummies(data=df_final['event']))

In [ ]:
# conversion en datetime de timestamp
df_final['timestamp'] = pd.to_datetime(df_final['timestamp'], unit='ms')
# Créer la Colonne Mois
df_final['month'] = df_final['timestamp'].dt.month

In [ ]:
display(df_final.head())
display(df_final.shape)

In [ ]:
#Vérification NaN, duplicated, types col
display(pd.DataFrame(index=df_final.columns, columns=['%_valeurs_manquantes'], data= (df_final.isna().sum().values / len(item)*100)))
print("Doublons")
display(df_final.duplicated().sum())
print("Types des variables")
display(df_final.dtypes)

In [ ]:
#Remplacement des fillna de parentid par 9999 qui correspond à Other
df_final['parentid'].fillna(9999, inplace=True)

In [ ]:
# Conversion des variables
df_final[['available','parentid','addtocart','transaction','view']]=df_final[['available','parentid','addtocart','transaction','view']].astype(int)


In [ ]:
#aperçu des doublons
duplicates_df_final = df_final[df_final.duplicated(keep=False)]
display(duplicates_df_final)
#suppression doublons car ils sont identiques
df_final=df_final.drop_duplicates()

In [ ]:
df_final=df_final.drop(['event','timestamp'], axis=1)

In [ ]:
display(df_final.head())
display(df_final.shape)

**Agrégation des données grâce au groupby sur les variables, application des fonctions d'agrégation différentes en fonction de chaque variable**

*   addtocart : somme
*   transaction : somme
*   view: somme
*   available :garder que la dernière valeur
*   categoryid":garder la première valeur
*   parentid": garder la première valeur
*   month: garder le nombre de mois unique
*   visitorid : garder le nombre de visiteurs uniques qui ont consulté le produit

In [ ]:
#agreger les données
dictag={'addtocart':'sum','transaction':'sum','view':'sum','available':'last','categoryid':'first','parentid':'first','month':'nunique','visitorid':'nunique' }
df_final_ag=df_final.groupby('itemid').agg(dictag)

# Quels principaux enseignements tirés de ce jeu de données?

**Aperçu du df_final qui nous servira pour les parties Visualisation et Modèles de Machine Learning**

In [ ]:
display(df_final_ag.head())
display(df_final_ag.shape)
#variable cible: Transaction

# Data Visualisation
**Comprendre les habitudes de navigation sur le site**
**Axe 1- une visualisation descriptive des données**
- Distribution du nbre de vu/ajout panier/transaction des produits
- Distribution du nbre de visiteurs pour un produit
- Quelle est la proportion de produits disponible/non disponible?
- le nbre de mois unique où les visiteurs consulte un produit?
- Quelles sont les 10 categ qui réalisent des events de vu/d'ajout panier/ de transaction des produits?

**Axe 2- Impacte des différents facteurs sur les events**
- Est-ce qu'il y a un lien entre la disponibilité ou non du produits et les vus?
- Relation entre le nbre de visiteur du produit et la transaction

**Axe 3- correlations entre les variables**
Correlation variables numériques

In [ ]:
#Rappel du type des variables
#variables numériques: addcart,transaction,view,visitorid, month
#variables catégorielles:available,category,parentid

In [ ]:
df_final_ag.describe()

**Descritpion univariée**

In [ ]:
#Distribution du nbre de vu/add/transaction des produits
#sur la période considérée, la plupart des produits ont été vu une fois, certains entre 2 et 5 fois
#Très peu ont été ajoutés et parmis ceux-ci, le nbre d'ajout de produits est de 1 voir 2
#Il en est de même pour les transactions, une seule transaction
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=3,
                   subplot_titles = ['Distribution des views','Distribution des ajouts paniers','Distribution des transactions'])

fig.add_trace(go.Histogram(x=df_final_ag['view'],
                           marker_color = 'blue',
                           name='View',
                           marker_line = dict(width = 1, color = 'white'),
                           xbins=dict( # bins used for histogram
                            start=0,
                             end=30,
                             size=1)),
                           1,1)

fig.add_trace(go.Histogram(x=df_final_ag['addtocart'],
                           marker_color = 'orange',
                           name='Ajout panier',
                           marker_line = dict(width = 1, color = 'white'),
                           xbins=dict(
                            start=0,
                             end=10,
                             size=1)),
                           1,2)

fig.add_trace(go.Histogram(x=df_final_ag['transaction'],
                           marker_color = 'green',
                           name='Transaction',
                           marker_line = dict(width = 1, color = 'white'),
                           xbins=dict(
                            start=0,
                             end=10,
                             size=0.5)),
                           1,3)


**Au regards de ces distributions, la plupart des produits ont été vu une fois, certains entre 2 et 5 fois.**

**Le nombre d'ajout au panier est d'un produit voir 2 est assez faible par rapport au non ajout.**

**Il en est de même pour les transactions, 74k de 0 transaction vs 1.6k de transacttion**

In [ ]:
#Distribution du nbre de visiteurs pour un produit
fig=go.Figure()
fig.add_trace(go.Histogram(x=df_final_ag['visitorid'],
                           marker_color = 'blue',
                           name='Nbre de visiteurs',
                           marker_line = dict(width = 1, color = 'white'),
                           xbins=dict( # bins used for histogram
                            start=0,
                             end=20,
                             size=1)))

**Beaucoup de visiteurs unique**

**Mais également un nombre non négligeable de 2 visiteurs ou plus sur un produit**

In [ ]:
values=df_final.month.value_counts()
values2=df_final_ag.month.value_counts()
from plotly.subplots import make_subplots
fig = make_subplots(rows=1, cols=2,
                    specs=[[{'type':'domain'}, {'type':'domain'}]],
                   subplot_titles = ['Répartition des mois','Nbre de mois unique'])

fig.add_trace(go.Pie(values=values.values,
                     labels=values.index,
                     pull = [0.1,0,0,0,0],
                     legendgroup="group1",
                     legendgrouptitle_text = " Mois"),
                     1,1)
fig.add_trace(go.Pie(values=values2.values,
                     labels=values2.index,
                     pull = [0.1,0,0,0,0],
                     legendgroup="group2",
                     legendgrouptitle_text = "Mois unique"),
                     1,2)
fig.show()
#la période de juin à juillet semble être la période qui a le plus d'evènement, contrairement au mois de septembre
#le nbre de mois unique où un produit est consulté ou acheté est de généralement de 1 voir 2 (donc un cycle d'achat relativement court)

**Sur la période considérée, les mois de juin et juillet semble être la période qui a le plus d'evènement, contrairement au mois de septembre**

**Le nbre de mois unique où un produit est consulté ou acheté est de généralement de 1 voir 2 (donc un cycle d'achat relativement court?/ou un produit qui n'est pas de 1ière nécessité?)**

**Difficile d'apporter un avis métier au regard de l'anonymisation des données**

In [ ]:
#La disponibilité des produits
values=df_final_ag.available.astype(str).value_counts()
fig=go.Figure([go.Bar(x=values.index,
                      y = values)])

fig.update_layout(title = 'La disponibilité des produits',
                 xaxis_title = 'Available',
                 yaxis_title = 'Count')

**Sur la période considérée, nous avons un peu plus de produits indisponibles en comparaison aux produits disponibles, ce qui a dû influencer la non réalisation des transactions sur le site**

In [ ]:
topview=df_final_ag.sort_values(by='view', ascending=False).head(10)
topadd=df_final_ag.sort_values(by='addtocart', ascending=False).head(10)
toptrans=df_final_ag.sort_values(by='transaction', ascending=False).head(10)
topview.categoryid=topview.categoryid.astype(str)
topadd.categoryid=topadd.categoryid.astype(str)
toptrans.categoryid=toptrans.categoryid.astype(str)

fig2 = make_subplots(rows=1, cols=3,
                   subplot_titles = ['Top catégorie view','Top catégorie ajouts paniers','Top catégorie transactions'])

fig2.add_trace(go.Bar(x=topview.categoryid,
                      y = topview['view'],
                      name='View'),
                      1,1)
fig2.add_trace(go.Bar(x=topadd.categoryid,
                      y = topadd['addtocart'],
                      name='Addtocart'),
                      1,2)
fig2.add_trace(go.Bar(x=toptrans.categoryid,
                      y = toptrans['transaction'],
                      name='Transaction'),
                      1,3)



**Le produit phare est le produit 1037 avec 26 transactions au total pour le produit qui a été acheté le plus**

**Description bivariée et multivariée**

In [ ]:
#relation entre le nbre de vus et les ajouts au panier
#relation entre les ajouts et les transactions
fig3 = make_subplots(rows=1, cols=3,
                   subplot_titles = ['Relation vus vs ajout panier','Relation ajout panier et transaction','Relation visiteurs et les vus'])

fig3.add_trace(go.Scatter(x=df_final_ag.view,
                          y = df_final_ag.addtocart,
                          mode = "markers"),
                          1,1)

fig3.add_trace(go.Scatter(x=df_final_ag.addtocart,
                          y = df_final_ag.transaction,
                          mode = "markers"),
                          1,2)
fig3.add_trace(go.Scatter(x=df_final_ag.visitorid,
                          y = df_final_ag.view,
                          mode = "markers"),
                          1,3)

In [ ]:
#relation linéaire entre le nbre de visiteur du produit et les vus
sns.lmplot(x='visitorid', y="view", data=df_final_ag, order = 2, line_kws = {'color': 'red'})
plt.title("Relation entre le nbre de visiteur unique et le nbre de vus")
plt.show()

In [ ]:
#relation linéaire entre le nbre de visiteur du produit et les ajouts
sns.lmplot(x='visitorid', y="addtocart", data=df_final_ag, order = 2, line_kws = {'color': 'red'})
plt.title("Relation entre le nbre de visiteur unique et le nbre d'ajouts")
plt.show()

In [ ]:
#relation linéaire entre le nbre de vu et le nbre d'ajout au panier=> sns.lmplot
sns.lmplot(x='view', y="addtocart", data=df_final_ag, order = 2, )
plt.title("Relation entre le nbre de view et ajoutés au panier")
plt.show()

In [ ]:
var_num=df_final_ag[['addtocart','transaction','view','visitorid', 'month']]

In [ ]:
#De façon logique, il y a une correlation entre le nbre de Il semble avoir une plus grande correlation entre
#correlation variables numerique()
var_num=df_final_ag[['addtocart','transaction','view','visitorid', 'month']]
x=var_num.corr()
fig, ax = plt.subplots(figsize = (12,12))
sns.heatmap(x, annot=True, ax=ax, cmap='plasma');

**De façon logique, il y a une correlation entre le nbre de visitor et le nombre de vus.**

**Celle entre le nombre de visiteurs et la transaction semble assez faible**

In [ ]:
sns.lmplot(x='view', y="addtocart", data=df_final_ag, order = 2, line_kws = {'color': 'red'})
plt.title("Relation entre le nbre de view et ajoutés au panier")
plt.show()

# Nous allons prodécer à présent à la méthode de clustering Kmeans pour obtenir des groupes homogènes qui ont des comportements similaires

In [ ]:
#clustering pour déterminer des groupes de personnes qui ont des comportements similaires.
#KMEANS
from sklearn.preprocessing import StandardScaler
 # Sélectionner uniquement les colonnes numériques
df_numeric = df_final_ag.drop(['available', 'categoryid','parentid'], axis=1)
df_normalized= df_numeric.copy()

# Normaliser les données
scaler = StandardScaler()
col=['addtocart', 'transaction' ,'view','month','visitorid' ]
df_normalized.loc[:,col]=scaler.fit_transform(df_normalized[col])
df_numeric.head()

In [ ]:
df_final_ag.head()
display(df_final_ag.shape)
display(df_numeric.shape)
display(df_normalized.shape)

In [ ]:
from sklearn.cluster import KMeans
from scipy.spatial.distance import cdist
from sklearn.metrics import silhouette_score, adjusted_rand_score
import warnings
warnings.filterwarnings('ignore')
distorsion=[]
K = range(2, 10)
for k in K:
    kmeanModel = KMeans(n_clusters=k, random_state=42)
    kmeanModel.fit(df_normalized)
    distorsion.append(sum(np.min(cdist(df_normalized, kmeanModel.cluster_centers_, 'euclidean'), axis=1)) / np.size(df, axis = 0))

In [ ]:
# Tracer la courbe de la distorsion en fonction du nombre de clusters
plt.plot(range(2, 10), distorsion, 'bx-')
plt.xlabel('Nombre de clusters')
plt.ylabel('Distorsion')
plt.title('Méthode du coude')
plt.show()

In [ ]:
#selon la méthode du coude, le nombre de cluster est de 3
#Entrainement de l'algorithme sur le df, et calcul des positions des K centroïdes et les labels
clf_kmean = KMeans(n_clusters = 3, random_state=42)
clf_kmean=clf_kmean.fit(df_normalized)
centroids = clf_kmean.cluster_centers_
label = clf_kmean.labels_
np.shape(label)


In [ ]:
df_final_ag['cluster_label'] = clf_kmean.labels_
# Interprétation des groupes
# Afficher les statistiques des clusters
cluster_stats = df_final_ag.groupby('cluster_label')[df_numeric.columns].mean()
cluster_stats

In [ ]:
#Répartition de nos clusters avec la méthodes des Kmeans
#Nous avons près de 60.000 observations dans le clusters n°1, nous avons très peu d'observation dans le cluster n°2
plt.hist(df_final_ag['cluster_label']);

In [ ]:
# Visualisation des groupes
sns.scatterplot(data=df_final_ag, x='transaction', y='visitorid', hue='cluster_label', palette='Set1')
plt.title("Clusters K-means")
plt.show()

In [ ]:
#le cluster n°1 est la cible potentielle pour nous car elle est composée de personnes suceptibles de réaliser des transactions

In [ ]:
#Affichage du df_final
display(df_final_ag.head())

In [ ]:

#Récuprer les categories de produits qui sont dans le clustern°1
dictag={'categoryid':'unique'}
categ_group=df_final_ag.groupby('cluster_label').agg(dictag)
categ_group.iloc[1,0]
#Nous avons dans le cluster n°1 146 categories de produits à potentiel

In [ ]:
df_final_ag_trans=df_final_ag.loc[df_final_ag['transaction']!=0]

In [ ]:
#Visualisation du nombre de transaction par cluster
#filtrage df avec au moins une transaction
df_final_ag_trans=df_final_ag.loc[df_final_ag['transaction']!=0]
# Créer une table de contingence entre cluster_label et transaction
contingency_table = pd.crosstab(df_final_ag_trans['cluster_label'], df_final_ag_trans['transaction'])

# Tracer le barplot
contingency_table.plot(kind='bar', stacked=True)

# Ajouter des étiquettes et des titres
plt.xlabel('Cluster')
plt.ylabel('Nombre de transaction')
plt.title('Répartition des transaction par cluster')

# Afficher la légende
plt.legend(title='Transaction')

# Afficher le graphique
plt.show()